In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

%matplotlib inline
plt.style.use('ggplot')

In [2]:
train_df = pd.read_csv('dataset/telstra/train.csv')
print 'train_df', train_df.shape
test_df = pd.read_csv('dataset/telstra/test.csv')
print 'test_df', test_df.shape
event_df = pd.read_csv('dataset/telstra/event_type.csv')
print 'event_df', event_df.shape
log_df = pd.read_csv('dataset/telstra/log_feature.csv')
print 'log_df', log_df.shape
resource_df = pd.read_csv('dataset/telstra/resource_type.csv')
print 'resource_df', resource_df.shape
severity_df = pd.read_csv('dataset/telstra/severity_type.csv')
print 'severity_df', severity_df.shape

train_df (7381, 3)
test_df (11171, 2)
event_df (31170, 2)
log_df (58671, 3)
resource_df (21076, 2)
severity_df (18552, 2)


In [3]:
# Set 'id' as index
train_df.set_index('id', drop=True, inplace=True)
test_df.set_index('id', drop=True, inplace=True)
event_df.set_index('id', drop=True, inplace=True)
log_df.set_index('id', drop=True, inplace=True)
resource_df.set_index('id', drop=True, inplace=True)
severity_df.set_index('id', drop=True, inplace=True)

In [4]:
# Drop target variable from training set
y = train_df['fault_severity']
train_df.drop('fault_severity', axis=1, inplace=True)
all_data = pd.concat([train_df, test_df])

In [5]:
# Categorize and create dummy variables for each categorical features
# Creating dummy variables will increase the number of features (since features are very less here).
all_data['location'] = all_data['location'].astype('category')
all_data = pd.get_dummies(all_data)
event_df['event_type'] = event_df['event_type'].astype('category')
event_df = pd.get_dummies(event_df)
resource_df['resource_type'] = resource_df['resource_type'].astype('category')
resource_df = pd.get_dummies(resource_df)
severity_df['severity_type'] = severity_df['severity_type'].astype('category')
severity_df = pd.get_dummies(severity_df)

In [6]:
# Remove duplicate ids
events = event_df.reset_index().groupby('id').agg(np.sum)
resources = resource_df.reset_index().groupby('id').agg(np.sum)
severities = severity_df.reset_index().groupby('id').agg(np.sum)

In [7]:
# Join every features into a single dataframe
all_data = pd.merge(all_data, events, left_index=True, right_index=True, how='inner')
all_data = pd.merge(all_data, resources, left_index=True, right_index=True, how='inner')
all_data = pd.merge(all_data, severities, left_index=True, right_index=True, how='inner')

In [8]:
# Split again into train and test dataframes
train = all_data.loc[train_df.index]
test = all_data.loc[test_df.index]

In [9]:
X_train, X_cv, y_train, y_cv = train_test_split(train, y, test_size=0.3, random_state=7)

In [ ]:
n_estimators = np.arange(5, 50)

In [21]:
print "classification report"
print classification_report(y_cv, y_cv_pred)
print "confusion matrix"
print confusion_matrix(y_cv, y_cv_pred)

classification report
             precision    recall  f1-score   support

          0       0.73      0.85      0.78      1401
          1       0.47      0.31      0.37       585
          2       0.45      0.38      0.41       229

avg / total       0.63      0.66      0.64      2215

confusion matrix
[[1193  156   52]
 [ 351  180   54]
 [  98   45   86]]


In [24]:
# This is how we read confusion matrix
pd.crosstab(y_cv, y_cv_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,2,All
True,,,,
0,1193,156,52,1401
1,351,180,54,585
2,98,45,86,229
All,1642,381,192,2215


In [61]:
important_feats = dtc.feature_importances_

In [74]:
important_feats_30 = np.sort(important_feats)[-30:]
important_feats_30


array([ 0.0040702 ,  0.00416019,  0.0042206 ,  0.00425749,  0.00431768,
        0.00443572,  0.0045372 ,  0.00478789,  0.00493397,  0.00512673,
        0.00520645,  0.00527635,  0.00538035,  0.00611191,  0.00617091,
        0.0062471 ,  0.00655645,  0.00747443,  0.00857645,  0.00902457,
        0.00954352,  0.01037422,  0.01066244,  0.0119002 ,  0.01195648,
        0.01273181,  0.01629774,  0.02464181,  0.05531908,  0.16515463])